In [12]:
from typing import Text, Any, Dict, List, Set, Union, Optional
from dataclasses import dataclass
import string

@dataclass
class token:
    value:str
    children:Optional[Dict[Text, 'token']]
    
class PatternFormatter(string.Formatter):

    def format_field(self, value, spec):
        if spec == '_':
            return value.value
        elif spec.startswith('/'):
            return value.children[spec[1:]].value
        else:
            return super(SuperFormatter, self).format_field(value, spec)
        
pf = PatternFormatter()
patt='behave {obj:_} for {obj:/obj}, modal {_:_}'
obj=token('eat', children={'obj':token('food',{})})
pf.format(patt, obj=obj, _=token('want',{}))

'behave eat for food, modal want'

In [15]:
objs={'obj':obj, '_':token('want',{})}
pf.format(patt, **objs)

'behave eat for food, modal want'

In [40]:
# $ str 'Rezervasyonumu onaylamak istiyorum.'
results=[{'delivery': 'sentence',
  'inspector': 'specs_of',
  'part': '_',
  'pattern': 'behave {obj:_} for {obj:/obj}, modal {_:_}',
  'provider': 'default',
  'value': {'category': 'request',
            'pos': 'v',
            'subs': [{'candidates': 'request',
                      'substitute': 'request',
                      'word': 'iste'}],
            'words': ['istiyorum/iste']}},
 {'delivery': 'slot',
  'inspector': 'pipes',
  'part': 'verb:obj/obj',
  'pattern': 'behave {obj:_} for {obj:/obj}, modal {_:_}',
  'provider': 'cat/cat_proc',
  'value': [{'cat': 'reservation',
             'path': '/obj/obj',
             'pos': 'noun',
             'trans': 'reservation',
             'value': 'reservation',
             'word': 'rezervasyon'}]},
 {'delivery': 'sentence',
  'inspector': 'kind_of',
  'part': 'obj',
  'pattern': 'behave {obj:_} for {obj:/obj}, modal {_:_}',
  'provider': 'default',
  'value': {'category': 'approve', 'pos': '*', 'word': 'onaylamak/onayla'}}]


results[0]

{'delivery': 'sentence',
 'inspector': 'specs_of',
 'part': '_',
 'pattern': 'behave {obj:_} for {obj:/obj}, modal {_:_}',
 'provider': 'default',
 'value': {'category': 'request',
  'pos': 'v',
  'subs': [{'candidates': 'request', 'substitute': 'request', 'word': 'iste'}],
  'words': ['istiyorum/iste']}}

In [41]:
all_pats=set([r['pattern'] for r in results if '{' in r['pattern']])
all_pats

{'behave {obj:_} for {obj:/obj}, modal {_:_}'}

In [42]:
for pat in all_pats:
    rs=[r for r in results if r['pattern']==pat]
    print(len(rs))

3


In [29]:
def get_descriptor(result, expr) -> List[Text]:
    from jsonpath_ng.ext import parse
    jsonpath_expr = parse(expr)
    return next((match.value for match in jsonpath_expr.find(result)), None)
get_descriptor(results[0], '$.value.category')

'request'

In [22]:
get_descriptor(results[1], '$.value[*].cat')

'reservation'

In [33]:
import re

descriptors={'specs_of': '$.value.category',
             'kind_of': '$.value.category',
             'pipes': '$.value[*].value',
            }

def set_val(var_map, part, val):
    if ':' in part:
        domain, part, sub=re.split('[:/]', part)
        if part not in var_map:
            var_map[part]=token('', children={sub:token(val,{})})
        else:
            var_map[part].children[sub]=token(val, {})
    else:
        if part not in var_map:
            var_map[part]=token(val, {})
        else:
            var_map[part].value=val

var_map={}
for rs in results:
    ins=rs['inspector']
    part=rs['part']
    if ins in descriptors:
        expr=descriptors[ins]
        val=get_descriptor(rs, expr)
        if val:
            set_val(var_map, part, val)
        else:
            print(f'absent val: {expr}')

var_map

{'_': token(value='request', children={}),
 'obj': token(value='approve', children={'obj': token(value='reservation', children={})})}

In [34]:
pf.format(patt, **var_map)

'behave approve for reservation, modal request'

In [32]:
import re
re.split('[:/]', 'verb:obj/obj')

['verb', 'obj', 'obj']